## Data Integration

In this section, we integrate the cleaned NOAA climate dataset with the cleaned EIA electricity retail sales dataset.  
The goal is to produce a unified monthly panel dataset containing climate variables (temperature, CDD, HDD) and electricity consumption/revenue for each state and month.

### 1. Load Cleaned Input Datasets

We begin by loading the cleaned outputs generated from the workflow:

- noaa_climate_clean.csv (state-level monthly climate indicators)
- eia_retail_sales_clean.csv (state-level monthly electricity sales and revenue)

Both datasets have already been cleaned in earlier steps, ensuring consistent variable types and the removal of invalid records.

### 2. Normalize Time Fields

The NOAA dataset already provides `year` and `month` as separate fields, while the EIA dataset stores time in a single `period` string formatted as "YYYY-MM".

We split `period` into:
- year (integer)
- month (integer)

This produces a common temporal structure across both datasets and supports precise merging.

### 3. Align State Identifiers

To ensure consistent joins between datasets:

- NOAA uses full state names.
- EIA uses two-letter state abbreviations (`stateid`).

Since NOAA also contains a mapping to state abbreviations, we align everything using the `stateid` field as the key.

This guarantees that each observation refers to the same geographic entity.

### 4. Merge NOAA and EIA

We perform an inner join on the shared keys:
- stateid
- year
- month

This ensures that only months with data in both datasets are included in the final integrated table.

### 5. Post-Merge Cleaning and Reordering

After merging, we:

- remove redundant fields (e.g., `period`)
- reorder columns into a logical and analysis-friendly schema:

year  
month  
stateid  
state_name  
tavg  
cdd  
hdd  
sales  
revenue  

The result is a tidy, well-structured monthly state-level dataset ready for modeling and visualization.

### 6. Integrated Dataset Overview

We inspect:

- missing values (none expected after cleaning)
- overall shape
- sample rows for sanity checking

The final integrated dataset serves as the foundation for our downstream analysis on how climate variation relates to electricity consumption patterns across U.S. states.

In [1]:
import pandas as pd

# Paths to cleaned datasets
noaa_clean_path = "../data/clean/noaa_climate_clean.csv"
eia_clean_path = "../data/clean/eia_retail_sales_clean.csv"

# Load cleaned datasets
noaa = pd.read_csv(noaa_clean_path)
eia = pd.read_csv(eia_clean_path)

# Quick preview
display(noaa.head())
display(eia.head())


,state_name,year,month,tavg,cdd,hdd
0,Alabama,1895,jan,43.1,5.0,717.0
1,Alabama,1896,jan,43.5,4.0,693.0
2,Alabama,1897,jan,41.8,3.0,752.0
3,Alabama,1898,jan,49.0,19.0,545.0
4,Alabama,1899,jan,43.8,5.0,690.0


,period,stateid,sales,revenue
0,2001-01,AK,521.03566,51.96404
1,2001-01,AL,7362.47302,407.61261
2,2001-01,AR,3804.21013,216.58535
3,2001-01,AZ,4786.79176,304.10688
4,2001-01,CA,21744.31668,1893.25678


In [2]:
print("NOAA year range:", noaa["year"].min(), "→", noaa["year"].max())
print("EIA period range:", eia["period"].min(), "→", eia["period"].max())


NOAA year range: 1895 → 2025
EIA period range: 2001-01 → 2025-09


In [3]:
# Split period into year and month
eia["year"] = eia["period"].str.slice(0,4).astype(int)
eia["month"] = eia["period"].str.slice(5,7).astype(int)

eia.head()


,period,stateid,sales,revenue,year,month
0,2001-01,AK,521.03566,51.96404,2001,1
1,2001-01,AL,7362.47302,407.61261,2001,1
2,2001-01,AR,3804.21013,216.58535,2001,1
3,2001-01,AZ,4786.79176,304.10688,2001,1
4,2001-01,CA,21744.31668,1893.25678,2001,1


In [ ]:
month_map = {
    "jan": 1, "feb": 2, "mar": 3, "apr": 4,
    "may": 5, "jun": 6, "jul": 7, "aug": 8,
    "sep": 9, "oct": 10, "nov": 11, "dec": 12
}

noaa["month"] = noaa["month"].map(month_map)

noaa.head()

,state_name,year,month,tavg,cdd,hdd
0,Alabama,1895,1,43.1,5.0,717.0
1,Alabama,1896,1,43.5,4.0,693.0
2,Alabama,1897,1,41.8,3.0,752.0
3,Alabama,1898,1,49.0,19.0,545.0
4,Alabama,1899,1,43.8,5.0,690.0


In [5]:
state_abbrev_map = {
    "Alabama": "AL", "Arizona": "AZ", "Arkansas": "AR", "California": "CA",
    "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE", "Florida": "FL",
    "Georgia": "GA", "Idaho": "ID", "Illinois": "IL", "Indiana": "IN",
    "Iowa": "IA", "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA",
    "Maine": "ME", "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI",
    "Minnesota": "MN", "Mississippi": "MS", "Missouri": "MO", "Montana": "MT",
    "Nebraska": "NE", "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ",
    "New Mexico": "NM", "New York": "NY", "North Carolina": "NC",
    "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK", "Oregon": "OR",
    "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC",
    "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT",
    "Vermont": "VT", "Virginia": "VA", "Washington": "WA",
    "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY"
}

noaa["stateid"] = noaa["state_name"].map(state_abbrev_map)


In [6]:
noaa[noaa["stateid"].isna()]["state_name"].unique()


array([], dtype=object)

In [7]:
min_eia_year = eia["year"].min()

noaa_aligned = noaa[noaa["year"] >= min_eia_year].copy()

In [ ]:
integrated_preview = noaa_aligned.merge(
    eia,
    on=["stateid", "year", "month"],
    how="inner"
)

integrated_preview.head()

,state_name,year,month,tavg,cdd,hdd,stateid,period,sales,revenue
0,Alabama,2001,1,41.2,3.0,770.0,AL,2001-01,7362.47302,407.61261
1,Alabama,2002,1,46.8,14.0,599.0,AL,2002-01,7120.43573,389.88061
2,Alabama,2003,1,40.0,2.0,811.0,AL,2003-01,7280.36274,429.20504
3,Alabama,2004,1,44.2,6.0,679.0,AL,2004-01,7521.61292,438.66058
4,Alabama,2005,1,49.2,20.0,532.0,AL,2005-01,7402.72800,420.52459


In [ ]:
print("=== Missing Value Summary (Integrated Dataset) ===")
missing_counts = integrated_preview.isna().sum()
missing_percents = integrated_preview.isna().mean() * 100

missing_df = pd.DataFrame({
    "missing_count": missing_counts,
    "missing_percent": missing_percents.round(2)
})

missing_df

=== Missing Value Summary (Integrated Dataset) ===


,missing_count,missing_percent
state_name,0,0.0
year,0,0.0
month,0,0.0
tavg,0,0.0
cdd,0,0.0
hdd,0,0.0
stateid,0,0.0
period,0,0.0
sales,0,0.0
revenue,0,0.0


In [ ]:
# First, create a copy so we don't overwrite accidentally
integrated_clean = integrated_preview.copy()

# Drop redundant column
integrated_clean = integrated_clean.drop(columns=["period"])

# Reorder columns into a clean, logical structure
ordered_cols = [
    "year",
    "month",
    "stateid",
    "state_name",
    "tavg",
    "cdd",
    "hdd",
    "sales",
    "revenue"
]

integrated_clean = integrated_clean[ordered_cols]

# Show the result
integrated_clean.head()

,year,month,stateid,state_name,tavg,cdd,hdd,sales,revenue
0,2001,1,AL,Alabama,41.2,3.0,770.0,7362.47302,407.61261
1,2002,1,AL,Alabama,46.8,14.0,599.0,7120.43573,389.88061
2,2003,1,AL,Alabama,40.0,2.0,811.0,7280.36274,429.20504
3,2004,1,AL,Alabama,44.2,6.0,679.0,7521.61292,438.66058
4,2005,1,AL,Alabama,49.2,20.0,532.0,7402.72800,420.52459
